<a href="https://colab.research.google.com/github/endophenotype/Spark/blob/main/Spark_%D0%A1%D0%BE%D1%80%D1%82%D0%B8%D1%80%D0%BE%D0%B2%D0%BA%D0%B0_%D0%BF%D0%BE_%D0%B8%D0%BD%D0%B4%D0%B5%D0%BA%D1%81%D1%83_%D0%B8_%D0%B7%D0%BD%D0%B0%D1%87%D0%B5%D0%BD%D0%B8%D1%8E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz
!tar xf spark-3.4.0-bin-hadoop3.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3"
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

In [ ]:
import findspark
findspark.init()

In [ ]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
import pyspark.pandas as ps

Создание серии pandas-on-Spark путем передачи списка значений, позволяя
API pandas в Spark создать целочисленный индекс по умолчанию:

In [ ]:
s = ps.Series([1, 3, 5, np.nan, 6, 8])
print(type(s))
print(s)

<class 'pyspark.pandas.series.Series'>
0    1.0
1    3.0
2    5.0
3    NaN
4    6.0
5    8.0
dtype: float64


Создание кадра данных pandas-on-Spark путем передачи набора объектов,
которые можно преобразовать в ряды.

In [ ]:
psdf = ps.DataFrame(
 {'a': [1, 2, 3, 4, 5, 6],
 'b': [100, 200, 300, 400, 500, 600],
 'c': ["one", "two", "three", "four", "five", "six"]},
 index=[10, 20, 30, 40, 50, 60])
print(type(psdf))
psdf

<class 'pyspark.pandas.frame.DataFrame'>


,a,b,c
10,1,100,one
20,2,200,two
30,3,300,three
40,4,400,four
50,5,500,five
60,6,600,six


Создание pandas DataFrame путем передачи массива numpy с индексом даты и
времени и помеченными столбцами:

In [ ]:
dates = pd.date_range('20130101', periods=6)
pdf = pd.DataFrame(np.random.randn(6, 4), index=dates,
columns=list('ABCD'))
print(type(pdf))
pdf

<class 'pandas.core.frame.DataFrame'>


,A,B,C,D
2013-01-01,0.314070,-0.692824,-0.099695,-0.352332
2013-01-02,2.059596,0.835948,0.835169,-0.636309
2013-01-03,-0.781449,0.107534,0.997069,0.063576
2013-01-04,-0.462132,-0.114443,2.330371,2.297337
2013-01-05,1.164568,-0.117555,0.594833,1.138423
2013-01-06,1.947992,-1.268541,0.073194,-0.006961


Теперь этот кадр данных pandas можно преобразовать в кадр данных pandason-Spark. Он выглядит и ведет себя так же, как DataFrame pandas.


In [ ]:
pdf = pd.DataFrame(np.random.randn(6, 4), index=dates,
columns=list('ABCD'))
psdf = ps.from_pandas(pdf)
print(type(psdf))
psdf

<class 'pyspark.pandas.frame.DataFrame'>


/content/spark-3.4.0-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


,A,B,C,D
2013-01-01,-0.814221,-0.184297,0.307478,-0.120932
2013-01-02,-0.300036,0.303448,-2.395153,-1.525261
2013-01-03,0.831222,-0.311197,-0.623138,0.641323
2013-01-04,-0.380377,-1.951432,-1.769315,1.097422
2013-01-05,-0.356805,-1.772349,-1.915302,-0.040536
2013-01-06,-0.946458,-0.205107,-0.544837,-1.578703


Создание Spark DataFrame из pandas DataFrame

In [ ]:
dates = pd.date_range('20130101', periods=6)
pdf = pd.DataFrame(np.random.randn(6, 4), index=dates,
columns=list('ABCD'))
spark = SparkSession.builder.getOrCreate()
sdf = spark.createDataFrame(pdf)
print(type(sdf))
sdf.show()


<class 'pyspark.sql.dataframe.DataFrame'>
+--------------------+--------------------+-------------------+-------------------+
|                   A|                   B|                  C|                  D|
+--------------------+--------------------+-------------------+-------------------+
|   1.466315287979996| -1.9114009252086537| 0.3735081400510273| 0.3194235643081175|
|  -0.604817548318114|  0.8077108867334667|-1.8215884203352855|-0.5739728090522077|
|-0.16470589547921918|  0.9342492514956033| 0.8131577634973838| 0.4542387184729648|
|0.010490913940606621| -1.5246654116789993|-0.6435431655084777|0.41695501635568005|
|   0.722587756255043|0.006161736994996647|-0.6126243511865711| 0.1631914129641181|
|  0.7747883536744892| -1.0678813193390868|0.35492731388256266|-1.6354102684750276|
+--------------------+--------------------+-------------------+-------------------+



Создание pandas-on-Spark DataFrame из Spark DataFrame

In [ ]:
psdf = sdf.pandas_api()
print(type(psdf))
psdf

<class 'pyspark.pandas.frame.DataFrame'>


,A,B,C,D
0,1.466315,-1.911401,0.373508,0.319424
1,-0.604818,0.807711,-1.821588,-0.573973
2,-0.164706,0.934249,0.813158,0.454239
3,0.010491,-1.524665,-0.643543,0.416955
4,0.722588,0.006162,-0.612624,0.163191
5,0.774788,-1.067881,0.354927,-1.635410


В настоящее время поддерживаются типы,
общие как для Spark, так и для pandas.

In [ ]:
spark = SparkSession.builder.getOrCreate()
sdf = spark.createDataFrame(pdf)
psdf = sdf.pandas_api()
psdf.dtypes

A    float64
B    float64
C    float64
D    float64
dtype: object

Отображение индекса, столбцов и базовых данных numpy

In [ ]:
print(psdf.index)
print(psdf.columns)
print(psdf.to_numpy())
print(psdf.describe())

Int64Index([0, 1, 2, 3, 4, 5], dtype='int64')
Index(['A', 'B', 'C', 'D'], dtype='object')


/content/spark-3.4.0-bin-hadoop3/python/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: `to_numpy` loads all data into the driver's memory. It should only be used if the resulting NumPy ndarray is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


[[ 1.46631529 -1.91140093  0.37350814  0.31942356]
 [-0.60481755  0.80771089 -1.82158842 -0.57397281]
 [-0.1647059   0.93424925  0.81315776  0.45423872]
 [ 0.01049091 -1.52466541 -0.64354317  0.41695502]
 [ 0.72258776  0.00616174 -0.61262435  0.16319141]
 [ 0.77478835 -1.06788132  0.35492731 -1.63541027]]
              A         B         C         D
count  6.000000  6.000000  6.000000  6.000000
mean   0.367443 -0.459304 -0.256027 -0.142596
std    0.755707  1.214736  0.963436  0.823512
min   -0.604818 -1.911401 -1.821588 -1.635410
25%   -0.164706 -1.524665 -0.643543 -0.573973
50%    0.010491 -1.067881 -0.612624  0.163191
75%    0.774788  0.807711  0.373508  0.416955
max    1.466315  0.934249  0.813158  0.454239


Данные
#########

Транспонирование
#########

Сортировка по индексу
#########

Сортировка по значению

In [ ]:
print(psdf)
print("#########")
print(psdf.T)
print("#########")
print(psdf.sort_index(ascending=False))
print("#########")
print(psdf.sort_values(by='B'))

          A         B         C         D
0  1.466315 -1.911401  0.373508  0.319424
1 -0.604818  0.807711 -1.821588 -0.573973
2 -0.164706  0.934249  0.813158  0.454239
3  0.010491 -1.524665 -0.643543  0.416955
4  0.722588  0.006162 -0.612624  0.163191
5  0.774788 -1.067881  0.354927 -1.635410
#########
          0         1         2         3         4         5
A  1.466315 -0.604818 -0.164706  0.010491  0.722588  0.774788
B -1.911401  0.807711  0.934249 -1.524665  0.006162 -1.067881
C  0.373508 -1.821588  0.813158 -0.643543 -0.612624  0.354927
D  0.319424 -0.573973  0.454239  0.416955  0.163191 -1.635410
#########
          A         B         C         D
5  0.774788 -1.067881  0.354927 -1.635410
4  0.722588  0.006162 -0.612624  0.163191
3  0.010491 -1.524665 -0.643543  0.416955
2 -0.164706  0.934249  0.813158  0.454239
1 -0.604818  0.807711 -1.821588 -0.573973
0  1.466315 -1.911401  0.373508  0.319424
#########
          A         B         C         D
0  1.466315 -1.911401  0.37350

Pandas API в Spark в основном использует значение np.nanдля представления отсутствующих данных.
По умолчанию он не включается в расчеты.
Удалим все строки с отсутствующими данными.
Заполнение недостающих данных цифрой 5

In [ ]:
pdf1 = pdf.reindex(index=dates[0:4], columns=list(pdf.columns) + ['E'])
pdf1.loc[dates[0]:dates[1], 'E'] = 1
psdf1 = ps.from_pandas(pdf1)
print(psdf1)
print("#########")
print(psdf1.dropna(how='any'))
print("#########")
print(psdf1.fillna(value=5))

/content/spark-3.4.0-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


                   A         B         C         D    E
2013-01-01  1.466315 -1.911401  0.373508  0.319424  1.0
2013-01-02 -0.604818  0.807711 -1.821588 -0.573973  1.0
2013-01-03 -0.164706  0.934249  0.813158  0.454239  NaN
2013-01-04  0.010491 -1.524665 -0.643543  0.416955  NaN
#########


/content/spark-3.4.0-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


                   A         B         C         D    E
2013-01-01  1.466315 -1.911401  0.373508  0.319424  1.0
2013-01-02 -0.604818  0.807711 -1.821588 -0.573973  1.0
#########
                   A         B         C         D    E
2013-01-01  1.466315 -1.911401  0.373508  0.319424  1.0
2013-01-02 -0.604818  0.807711 -1.821588 -0.573973  1.0
2013-01-03 -0.164706  0.934249  0.813158  0.454239  5.0
2013-01-04  0.010491 -1.524665 -0.643543  0.416955  5.0


/content/spark-3.4.0-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


Описательная статистика:

In [ ]:
psdf1 = ps.from_pandas(pdf1)
print(psdf1)
print("#########")
print(psdf1.mean())

/content/spark-3.4.0-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


                   A         B         C         D    E
2013-01-01  1.466315 -1.911401  0.373508  0.319424  1.0
2013-01-02 -0.604818  0.807711 -1.821588 -0.573973  1.0
2013-01-03 -0.164706  0.934249  0.813158  0.454239  NaN
2013-01-04  0.010491 -1.524665 -0.643543  0.416955  NaN
#########
A    0.176821
B   -0.423527
C   -0.319616
D    0.154161
E    1.000000
dtype: float64


In [ ]:
Объединение результатов в структуру данных

In [ ]:
psdf = ps.DataFrame({'A': ['foo', 'bar', 'foo', 'bar',
 'foo', 'bar', 'foo', 'foo'],
 'B': ['one', 'one', 'two', 'three',
 'two', 'two', 'one', 'three'],
 'C': np.random.randn(8),
 'D': np.random.randn(8)})
psdf

,A,B,C,D
0,foo,one,-0.163064,-2.675978
1,bar,one,-2.011391,-0.638548
2,foo,two,1.049836,-0.579499
3,bar,three,-0.908411,0.521396
4,foo,two,1.552550,0.228866
5,bar,two,-1.146576,-0.569629
6,foo,one,1.148161,-0.055125
7,foo,three,0.658542,0.787660


In [ ]:
Группировка и последующее применение функции sum() к полученным
группам

In [ ]:
print(psdf.groupby('A').sum())

            C         D
A                      
bar -4.066378 -0.686781
foo  4.246023 -2.294076


Группировка по нескольким столбцам формирует иерархический индекс, и мы снова можем применить функцию суммирования.


In [ ]:
print(psdf.groupby(['A', 'B']).sum())

                  C         D
A   B                        
foo one    0.985096 -2.731103
    two    2.602386 -0.350633
bar three -0.908411  0.521396
    one   -2.011391 -0.638548
foo three  0.658542  0.787660
bar two   -1.146576 -0.569629


In [ ]:
pser = pd.Series(np.random.randn(1000),
 index=pd.date_range('1/1/2000', periods=1000))
psser = ps.Series(pser)
psser = psser.cummax()
psser.plot()

/content/spark-3.4.0-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:251: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


In [ ]:
pdf = pd.DataFrame(np.random.randn(1000, 4), index=pser.index,
 columns=['A', 'B', 'C', 'D'])
psdf = ps.from_pandas(pdf)
psdf = psdf.cummax()
psdf.plot()

/content/spark-3.4.0-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:251: FutureWarning:

Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead

